In [341]:
import numpy as np
import re
import pymorphy2
import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack, vstack
from scipy.sparse import csc_matrix
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
import warnings

pd.set_option('display.max_columns', 80) 
pd.set_option('display.max_rows', 100) 
%matplotlib inline
warnings.filterwarnings("ignore")

In [2]:
data_train = pd.read_csv("../data/avito/train.csv", sep=';')
data_test = pd.read_csv("../data/avito/test.csv", sep=';')

In [ ]:
morph = pymorphy2.MorphAnalyzer()
min_data = datetime.datetime.strptime(data_train.start_time.min(),  "%Y-%m-%d %H:%M:%S")

parse - приведение слов к нормальной форме с использованием pymorphy2

get_time - количество секунд от минимальной даты в выборке

In [219]:
def parse(x):
    return ' '.join([morph.normal_forms(i)[0] for i in re.findall('\w+', x)]) 
def rmse(x, y):
    return np.sqrt(np.mean((x-y)**2))
def get_time(x):
    return (datetime.datetime.strptime(x,  "%Y-%m-%d %H:%M:%S") - min_data).total_seconds()

preprocessing - предобработка данных. Заполняем nanы некоторой строкой. Категориальные кодируем label endcoderом. Ключевой признак сразу логарифмируем. Добавляем новые фичи: для каждого категориального признака ставим в соответствии каждому значению его частоту. 

In [221]:
def preprocessing(train, test):
    
    X = train.copy().fillna('na')
    Xtest = test.copy().fillna('na')
    
    X.title = X.title.apply(parse)
    Xtest.title = Xtest.title.apply(parse)
    
    X.start_time = X.start_time.apply(get_time)
    Xtest.start_time = Xtest.start_time.apply(get_time)
    
    data = pd.concat([X, Xtest])
    encoder = LabelEncoder()
    for col in ['owner_type', 'category', 'subcategory', 'param1', 'param2', 'param3',  'region']:
        encoder.fit(data[col])
        X[col] = encoder.transform(X[col])
        Xtest[col] = encoder.transform(Xtest[col])
        
    y = np.log(X.item_views + 1)
    X = X.drop(['item_views', '\ufeffid', 'item_id'], axis=1)
    Xtest = Xtest.drop(['\ufeffid', 'item_id'], axis=1)
    
    data = pd.concat([X, Xtest])
    for col in X.columns[1:]:
        col_map = data.groupby(col).apply(len)
        X['frequence_' + col] = X[col].map(col_map)
        Xtest['frequence_' + col] = Xtest[col].map(col_map)

    return X, Xtest, y

In [387]:
%%time
X, Xtest, y = preprocessing(data_train, data_test)

CPU times: user 10min 55s, sys: 1.71 s, total: 10min 56s
Wall time: 10min 57s


Разбиваем на выборку для обучения и валидации:

In [223]:
Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size=0.2, random_state=241)

Для категориального признака важен порядок, с которым мы обозначаем категории числами. Иногда полезно делать так, чтобы порядок выбирался в соответствии со средним значением ключевого признака внутри каждой категории. Важно, что можно использовать только таргеты из обучающей выборки, поэтому если категорий много или количество объектов внутри категории мало (то есть если средние значение внутри категории не стабильно), то это приведет к переобучению. И наоборот, если категорий мало, то это просто ничего не изменит. В данном случае полезно было переобозначить категории признака param1.

In [224]:
y_map = ytrain.groupby(Xtrain.param1).mean()
Xtrain['y'] = Xtrain.param1.map(y_map)
Xval['y'] = Xval.param1.map(y_map).fillna(ytrain.mean())

Удаление шумовых объектов.

Делаем кросс-валидацию. Для каждого разбиения X1, X2 строим модель на X1, прогнозируем на X2, смотрим ошибку. В итоге для каждого объекта есть ошибка. Удаляем объекты, ошибка на которых больше некоторого порога.

In [225]:
cv = KFold(len(Xtrain), n_folds=5, random_state=241, shuffle=True)
p = pd.Series(index=Xtrain.index)
model = xgboost.XGBRegressor(n_estimators=800, max_depth=8, seed=241)
for train, test in cv:
    X1 = Xtrain.iloc[train].drop('title', axis=1)
    X2 = Xtrain.iloc[test].drop('title', axis=1)
    y1 = ytrain.iloc[train]
    model.fit(X1, y1)
    p.iloc[test] = model.predict(X2)
index = abs(p - ytrain) < 2.5

In [226]:
Xtrain = Xtrain[index]
ytrain = ytrain[index]

Берем 5000 наиболее популярных слов из title. Делаем 5000 бинарных признаков - есть ли данное слово в title. Очень удобно делать с использованием CountVectorizer.

In [239]:
vectorizer = CountVectorizer(max_features=5000, binary=True)
vectorizer.fit(pd.concat([Xtrain, Xval]).title)
title1 = vectorizer.transform(Xtrain.title)
title2 = vectorizer.transform(Xval.title)

In [240]:
Xtrain2 = hstack([Xtrain.drop('title', axis=1), title1])
Xval2 = hstack([Xval.drop('title', axis=1), title2])

Три хгбуста с разной глубиной деревьев:

In [265]:
model1 = xgboost.XGBRegressor(n_estimators=5627, 
                              max_depth=10, 
                              subsample=0.9, 
                              colsample_bytree=0.8, 
                              reg_lambda=0, 
                              reg_alpha=2, 
                              learning_rate=0.05, 
                              seed=250)

model2 = xgboost.XGBRegressor(n_estimators=4511, 
                              max_depth=12, 
                              subsample=0.9, 
                              colsample_bytree=0.8, 
                              reg_lambda=0, 
                              reg_alpha=2, 
                              learning_rate=0.05, 
                              seed=200)


model3 = xgboost.XGBRegressor(n_estimators=1627, 
                              max_depth=20, 
                              subsample=0.9, 
                              colsample_bytree=0.8, 
                              reg_lambda=0, 
                              reg_alpha=2, 
                              learning_rate=0.05, 
                              seed=100)

Обучаем, смотрим скор:

In [266]:
%%time
model1.fit(Xtrain2, ytrain)
model2.fit(Xtrain2, ytrain)
model3.fit(Xtrain2, ytrain)

CPU times: user 6h 15min 20s, sys: 8min 35s, total: 6h 23min 55s
Wall time: 1h 13min 52s


In [268]:
p1 = model1.predict(Xval2)
p2 = model2.predict(Xval2)
p3 = model3.predict(Xval2)

In [270]:
rmse(yval, p1), rmse(yval, p2), rmse(yval, p3)

(0.5459918192097577,
 0.54581715413917054,
 0.54674203841925728,
 0.54825069853687303)

Подбираем коэффициенты блендинга:

In [390]:
a = np.array([5, 4, 4])
a = a / a.sum()
rmse(yval, a.dot([p1, p2, p3]))

0.54392735884533139

Проделываем ту же работу уже для финального X, Xtest.

In [391]:
y_map = y.groupby(X.param1).mean()
X['y'] = X.param1.map(y_map)
Xtest['y'] = Xtest.param1.map(y_map).fillna(y.mean())

In [393]:
%%time
cv = KFold(len(X), n_folds=5, random_state=241, shuffle=True)
p = pd.Series(index=X.index)
model = xgboost.XGBRegressor(n_estimators=800, max_depth=8, seed=241)
for train, test in cv:
    X1 = X.iloc[train].drop('title', axis=1)
    X2 = X.iloc[test].drop('title', axis=1)
    y1 = y.iloc[train]
    model.fit(X1, y1)
    p.iloc[test] = model.predict(X2)
index = abs(p - y) < 2.5

CPU times: user 46min 49s, sys: 58.1 s, total: 47min 47s
Wall time: 8min 28s


In [394]:
X = X[index]
y = y[index]

In [395]:
vectorizer.fit(pd.concat([X, Xtest]).title)
title1 = vectorizer.transform(X.title)
title2 = vectorizer.transform(Xtest.title)

In [396]:
X = hstack([X.drop('title', axis=1), title1])
Xtest = hstack([Xtest.drop('title', axis=1), title2])

In [397]:
%%time
model1.fit(X, y)
model2.fit(X, y)
model3.fit(X, y)

CPU times: user 5h 26min 41s, sys: 6min 7s, total: 5h 32min 48s
Wall time: 1h 5min 26s


In [398]:
p1test = model1.predict(Xtest)
p2test = model2.predict(Xtest)
p3test = model3.predict(Xtest)

In [399]:
ptest = a.dot([p1test, p2test, p3test])

In [400]:
answer = np.e** ptest - 1
sub = pd.DataFrame(answer).reset_index()
sub.columns = ['id', 'item_views']
sub.to_csv('C.csv', index=False, sep=';')

Говорят, можно обучить модель на трейне и на тесте, используя предсказания на тесте и тем самым немного улучшить результат. В данной задаче не прокатило, да и вообще для задачи регрессии это кажется дико. Но код оставил.

In [416]:
%%time
model1.fit(vstack([X, Xtest]), np.hstack([y, p1test]))
model2.fit(vstack([X, Xtest]), np.hstack([y, p2test]))
model3.fit(vstack([X, Xtest]), np.hstack([y, p3test]))

CPU times: user 11h 55min 17s, sys: 7min 24s, total: 12h 2min 42s
Wall time: 2h 24min 29s


In [417]:
p1test_ = model1.predict(Xtest)
p2test_ = model2.predict(Xtest)
p3test_ = model3.predict(Xtest)

In [418]:
ptest_ = a.dot([p1test_, p2test_, p3test_])

In [419]:
answer = np.e** ptest_ - 1
sub = pd.DataFrame(answer).reset_index()
sub.columns = ['id', 'item_views']
sub.to_csv('C2.csv', index=False, sep=';')